# Hyperparameteroptimization

Dataset: https://www.openml.org/search?type=data&sort=runs&status=active&id=1464
Task: https://www.openml.org/search?type=task&id=10101&source_data.data_id=1464

Steps:
   1. Definition of the data
   2. Definition of the hyperparameter space
   3. Loop over all different combinations of the hyperparamter space
       1. Define the model with the hyperparameters
       2. Optimize model (learning phase)
       3. Evaluate model and store metric with the parameters
 

## 0. Imports & utils

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from sklearn import svm, metrics
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score

import timeit

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import itertools

from mpl_toolkits import mplot3d

from scipy.io import arff
import pandas as pd

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import time, sys
from IPython.display import clear_output

import pysgpp


def update_progress(progress, time, remaining_time):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    text += "\nCurrent time per iteration: " + str(time)
    text += "\nApprox. time remaining: " + str(remaining_time)
    print(text)

    
def to_standard(lower, upper, value):
    return (value-lower)/(upper-lower)


def from_standard(lower, upper, value):
    return value*(upper-lower)+lower

## 1. Definition of Data

In [3]:
SPLIT_RATIO = 0.8

data = arff.loadarff('php0iVrYT.arff')
df = pd.DataFrame(data[0])

df['Class'].replace([b'1', b'2'], [1, 2], inplace=True)

X = []
Y = []

for i in range(len(df)):
    data = []
    data.append(df['V1'][i])
    data.append(df['V2'][i])
    data.append(df['V3'][i])
    data.append(df['V4'][i])
    
    X.append(data)
    Y.append(df['Class'][i])

print("Number of samples: ", len(X))

X, Y = shuffle(X, Y)
X = np.array(X[:740])
Y = np.array(Y[:740])

# 10 fold validation:
size_chunks = int(len(X)/10)
X_folds = [X[x:x+size_chunks] for x in range(0, len(X)-size_chunks, size_chunks)]
Y_folds = [Y[x:x+size_chunks] for x in range(0, len(Y)-size_chunks, size_chunks)]

#X_train = torch.Tensor(X[:int(len(X) * SPLIT_RATIO)])
#X_test = torch.Tensor(X[int(len(X) * SPLIT_RATIO):])
#Y_train = torch.Tensor(Y[:int(len(Y) * SPLIT_RATIO)])
#Y_test = torch.Tensor(Y[int(len(Y) * SPLIT_RATIO):])

#print("Number of training samples: ", len(X_train))
#print("Number of testing samples: ", len(X_test))

Number of samples:  748


## 2. Definition of Hyperparameter space

In [4]:
hyperparameters = {
    "C" : ["interval", 0.0, 10],
    "kernel" : ["set", "linear", "poly"],#, "rbf", "sigmoid", "precomputed"],
    "degree" : ["interval", 1, 5],
    "gamma" : ["set", "scale", "auto"],
    "tol" : ["interval", 0.000001, 0.1]
}

print("Number of hyperparameters: ", len(hyperparameters))

Number of hyperparameters:  5


## 3. Loop over combinations

In [5]:
number_per_dimension = 2

list_of_values = []

for k in hyperparameters.keys():
    
    liste = hyperparameters[k]
    
    values = []
    
    # fill with distinct values in interval (regular distance)
    if liste[0] == "interval":
        lower, upper = liste[1], liste[2]

        for j in range(number_per_dimension):
            values.append(lower+((upper-lower)/(2*number_per_dimension))+j*((upper-lower)/(number_per_dimension)))

        list_of_values.append(values)
        
    # discrete values already given because of set 
    else:
        for j in range(len(liste)-1):
            values.append(liste[j+1])
            
        list_of_values.append(values)
        

results_accuracy = []
results_loss = []

all_combinations = itertools.product(*list_of_values)


number_combinations = 1
for i in range(len(list_of_values)):
    number_combinations *= len(list_of_values[i])

    
time = 0
count = 0
for combination in all_combinations:
    #print("Current combination: ",combination)
    starttime = timeit.default_timer()
    
    C = combination[0]
    kernel = combination[1]
    degree = combination[2]
    gamma = combination[3]
    tol = combination[4]
    
    # 10 fold cross validation
    #accuracy_list = []
    #for i in range(10):
    """
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []

    for j in range(10):
        if (j == i):
            X_test.append(X[j])
            Y_test.append(Y[j])
        else:
            X_train.append(X[j])
            Y_train.append(Y[j])
    """  
    classifier = svm.SVC(C=C, kernel=kernel, degree=int(degree), gamma=gamma, tol=tol)
    #classifier.fit(X_train, Y_train)
    scores = cross_val_score(classifier, X, Y, cv=10)

    # Y_predicted = classifier.predict(X_test)

    #accuracy = metrics.accuracy_score(Y_test, Y_predicted)
    #accuracy_list.append(accuracy)
    accuracy = scores.mean()
    #accuracy_list.append(accuracy)
        
        
    #results_accuracy.append((combination, (sum(accuracy_list)/len(accuracy_list))))
    results_accuracy.append((combination, accuracy))
    count+=1
    percentage = count/number_combinations

    endtime = timeit.default_timer()
    time += (endtime-starttime)
    
    remaining_time_prediction = (time/count)*number_combinations - time
    
    update_progress(percentage, (endtime-starttime), remaining_time_prediction)
    
#print("Resulting losses: ")
#print(results_loss)
print("Iterations took ", time, " seconds.")

#print("Resulting accuracy: ")
#print(results_accuracy)

KeyboardInterrupt: 

## 4. Plotting and Result

In [ ]:
accuracies = [x[1] for x in results_accuracy]

print("Highest Accuracy: ", max(accuracies))
combi = -1
for i in results_accuracy:
    if i[1] == max(accuracies):
        combi = i[0]
print("With hyperparameter combination: ", combi)
